RGB Image Classification for Color Palette Generation

In [1]:
import turtle
from math import sqrt
import random

try:
    import Image
except ImportError:
    from PIL import Image

In [2]:
class Point:

    def __init__(self, coordinates):
        self.coordinates = coordinates

In [3]:
class Cluster:

    def __init__(self, center, points):
        self.center = center
        self.points = points


In [4]:
class KMeans:

    def __init__(self, n_clusters, min_diff=1):
        self.n_clusters = n_clusters
        self.min_diff = min_diff

    def calculate_center(self, points):
        n_dim = len(points[0].coordinates)
        vals = [0.0 for i in range(n_dim)]
        for p in points:
            for i in range(n_dim):
                vals[i] += p.coordinates[i]
        coords = [(v / len(points)) for v in vals]
        return Point(coords)

    def assign_points(self, clusters, points):
        plists = [[] for i in range(self.n_clusters)]

        for p in points:
            smallest_distance = float('inf')

            for i in range(self.n_clusters):
                distance = euclidean(p, clusters[i].center)
                if distance < smallest_distance:
                    smallest_distance = distance
                    idx = i

            plists[idx].append(p)

        return plists

    def fit(self, points):
        clusters = [Cluster(center=p, points=[p]) for p in random.sample(points, self.n_clusters)]

        while True:

            plists = self.assign_points(clusters, points)

            diff = 0

            for i in range(self.n_clusters):
                if not plists[i]:
                    continue
                old = clusters[i]
                center = self.calculate_center(plists[i])
                new = Cluster(center, plists[i])
                clusters[i] = new
                diff = max(diff, euclidean(old.center, new.center))

            if diff < self.min_diff:
                break

        return clusters

In [5]:
def euclidean(p, q):
    n_dim = len(p.coordinates)
    return sqrt(sum([
        (p.coordinates[i] - q.coordinates[i]) ** 2 for i in range(n_dim)
    ]))


In [6]:
def get_points(image_path):
    img = Image.open(image_path)
    img.thumbnail((200, 400))
    img = img.convert("RGB")
    w, h = img.size

    points = []
    for count, color in img.getcolors(w * h):
        for _ in range(count):
            points.append(Point(color))

    return points


In [7]:
def rgb_to_hex(rgb):
    return '#%s' % ''.join(('%02x' % p for p in rgb))

In [8]:
def get_colors(filename, n_colors=3):
    points = get_points(filename)
    clusters = KMeans(n_clusters=n_colors).fit(points)
    clusters.sort(key=lambda c: len(c.points), reverse=True)
    rgbs = [map(int, c.center.coordinates) for c in clusters]
    return list(map(rgb_to_hex, rgbs))

In [9]:
# colors = get_colors('flowers.jpg', n_colors=5)

# print(", ".join(colors))

# window = turtle.Screen()

# image = Image.open('flowers.jpg')
# image.show()

# turtle.hideturtle()
# turtle.speed(0)
# turtle.penup()
# turtle.setposition(-250, 0)
# for i in colors:
#     turtle.color(i, i)
#     turtle.pensize(5)
#     turtle.begin_fill()
#     turtle.circle(50)
#     turtle.end_fill()
#     turtle.penup()
#     turtle.forward(120)
#     turtle.pendown()
# turtle.getcanvas().postscript(file="test.eps")
# window.exitonclick()


In [10]:
import anvil.server
anvil.server.connect("server_47CLWJI76W43QF3GUHLKACRE-GYIBEZQJSM2FE2WA")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Debug for tommyt323@hotmail.com" as SERVER


In [12]:
import anvil.media

@anvil.server.callable
def classify_image(file):
    with anvil.media.TempFile(file) as filename:

        color = get_colors(filename, n_colors=5)

        return (", ".join(color))
    

In [14]:
%load_ext watermark

# python, ipython, packages, and machine characteristics
%watermark -v -m -p PIL,pandas,numpy,watermark 

# date
print (" ")
%watermark -u -n -t -z 

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Python implementation: CPython
Python version       : 3.9.7
IPython version      : 7.29.0

PIL      : 8.4.0
pandas   : 1.3.4
numpy    : 1.20.3
watermark: 2.3.0

Compiler    : Clang 10.0.0 
OS          : Darwin
Release     : 21.2.0
Machine     : x86_64
Processor   : i386
CPU cores   : 10
Architecture: 64bit

 
Last updated: Thu Apr 14 2022 23:59:41PDT

